<a href="https://colab.research.google.com/github/sarkar-sayan/URL-Classification/blob/main/POC_1_URL_Classification_Close2Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Code as a User based menu-driven

In [ ]:
pip install langdetect googletrans==4.0.0-rc1

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from random import choice
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
import string
from langdetect import detect
from googletrans import Translator
# Initialize Google Translator
translator = Translator()
# Initialize Tokenizer
vectorizer = TfidfVectorizer()
# Download the stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

  # List of user-agents for rotation
user_agents = [
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
     'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
     'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0',
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81',
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0',]

global productive_keywords
global non_productive_keywords

productive_keywords = ['study', 'research', 'education', 'work', 'project', 'python']
non_productive_keywords = ['game', 'social', 'fun', 'entertainment', 'video']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Enter or delete pre-saved Keywords

def keyword_management(productive_keywords, non_productive_keywords):
  choice = int(input("Enter choice to manage keywords (1 or 2) : 1. Productive  2. Non-productive  3. Display All -> "))
  if(choice == 1):
    get_productive(productive_keywords)
  elif(choice == 2):
    get_nonproductive(non_productive_keywords)
  elif choice not in [1, 2]:
    print("Displaying All....")
    print("All Productive Keywords:", ", ".join(productive_keywords))
    print("All Non-productive Keywords:", ", ".join(non_productive_keywords))
  return productive_keywords, non_productive_keywords


def get_productive(productive_keywords):
  ch = int(input("Enter choice to manage productive keywords (1 or 2) : 1. Add new  2. Delete old  3. Display all->"))
  if(ch == 1):
      c = int(input("Add NEW -> Operation mode: 1. Single  2. Bulk : "))
      if(c == 1):
        key = input("Enter new Productive Keyword: ")
        productive_keywords.append(key)
      if(c == 2):
        while True:
          key = input("\nEnter keyword to add (or 'done' to finish): ")
          if key.lower() == "done":
            break
          productive_keywords.append(key)
  elif(ch == 2):
      c = int(input("Delete OLD -> Operation mode: 1. Single  2. Bulk : "))
      if(c == 1):
        key = input("Enter Productive Keyword to delete: ")
        productive_keywords.remove(key)
      if(c == 2):
        while True:
          print(f"\nCurrent List: {productive_keywords}")
          key = input("Enter keyword to remove (or 'done' to finish): ")
          if key.lower() == "done":
            break
          try:
            productive_keywords.remove(key)
          except ValueError:
            print(f"'{key}' not found in the list. Try again.")
        print("\nFinal List: ", productive_keywords)
  elif(ch == 3):
      print("All Productive Keywords: ", ", ".join(productive_keywords))


def get_nonproductive(non_productive_keywords):
  ch = int(input("Enter choice to manage non-productive keywords (1 or 2) : 1. Add new  2. Delete old  3. Display all->"))
  if(ch == 1):
      c = int(input("Add NEW -> Operation mode: 1. Single  2. Bulk : "))
      if(c == 1):
        key = input("Enter new Non-productive Keyword: ")
        non_productive_keywords.append(key)
      if(c == 2):
        while True:
          key = input("\nEnter keyword to add (or 'done' to finish): ")
          if key.lower() == "done":
            break
          non_productive_keywords.append(key)
  elif(ch == 2):
      c = int(input("Delete OLD -> Operation mode: 1. Single  2. Bulk : "))
      if(c == 1):
        key = input("Enter Non-productive Keyword to delete: ")
        non_productive_keywords.remove(key)
      if(c == 2):
        while True:
          print(f"\nCurrent List: {non_productive_keywords}")
          key = input("Enter keyword to remove (or 'done' to finish): ")
          if key.lower() == "done":
            break
          try:
            non_productive_keywords.remove(key)
          except ValueError:
            print(f"'{key}' not found in the list. Try again.")
        print("\nFinal List:", non_productive_keywords)
  elif(ch == 3):
      print("All Non-productive Keywords: ", ", ".join(non_productive_keywords))



In [ ]:
# Function to get metadata from URL
def get_metadata_from_url(url):
    try:
        headers = {'User-Agent': choice(user_agents)}
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('title').get_text() if soup.find('title') else 'No title'

        description = soup.find('meta', attrs={'name': 'description'})
        if description:
            description = description.get('content')
        else:
            description = soup.find('meta', attrs={'property': 'og:description'})
            description = description.get('content') if description else 'No description'

        image = soup.find('meta', attrs={'property': 'og:image'})
        if image:
            image = image.get('content')
        else:
            image = soup.find('link', attrs={'rel': 'image_src'})
            image = image.get('href') if image else 'No image'

        text = ' '.join(p.get_text() for p in soup.find_all('p'))

        return {
            "title": title,
            "description": description,
            "image": image,
            "url": url,
            "text": text
        }
    except requests.RequestException as e:
        return {
            "title": "Error",
            "description": str(e),
            "image": "No image",
            "url": url,
            "text": ""
        }

# Preprocess text function
def preprocess_text(text):
    if not text:
        return ""
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Translate text if not in English
def translate_text_if_needed(text):
    try:
        language = detect(text)
        if language != 'en':
            translated = translator.translate(text, dest='en')
            return translated.text
        return text
    except Exception as e:
        return text

# Preprocess metadata function
def preprocess_metadata(metadata):
    title = translate_text_if_needed(metadata.get('title', ''))
    description = translate_text_if_needed(metadata.get('description', ''))
    text = translate_text_if_needed(metadata.get('text', ''))
    clean_title = preprocess_text(title)
    clean_description = preprocess_text(description)
    clean_text = preprocess_text(text)
    combined_clean_content = f"{clean_title} {clean_description} {clean_text}".strip()
    return combined_clean_content

# Function to extract domain from URL
def extract_domain(url):
    return url.split('//')[-1].split('/')[0]

# Keyword matching function
def count_keywords(text, keywords):
    tokens = text.split()
    return sum(token in keywords for token in tokens)

# Function to create feature matrix
def create_feature_matrix(df):
    tfidf_matrix = vectorizer.fit_transform(df['clean_content'])

    keyword_counts = df[['productive_keyword_count', 'non_productive_keyword_count']].values
    return np.hstack((tfidf_matrix.toarray(), keyword_counts))

In [ ]:
def prepare_dataset(productive_keywords, non_productive_keywords, dataset_url):
  data = pd.read_csv(dataset_url)
  data.drop(data.columns.difference(['url', 'label']), axis=1, inplace=True)
  # Scrape Content and Metadata
  data['metadata'] = data['url'].apply(get_metadata_from_url)
  # Apply Preprocessing to Metadata
  data['clean_content'] = data['metadata'].apply(preprocess_metadata)
  data['domain'] = data['url'].apply(extract_domain)
  data['productive_keyword_count'] = data['clean_content'].apply(lambda x: count_keywords(x, productive_keywords))
  data['non_productive_keyword_count'] = data['clean_content'].apply(lambda x: count_keywords(x, non_productive_keywords))
  return data

In [ ]:
def train_test_model(data):
  X = create_feature_matrix(data)
  y = data['label']

  # Train/Test Split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

  # Train Model
  model = MultinomialNB()
  model.fit(X_train, y_train)

  # Evaluate Model
  y_pred = model.predict(X_test)
  print("Accuracy:", accuracy_score(y_test, y_pred))
  #print(classification_report(y_test, y_pred))
  #print(classification_report(y_test, y_pred, target_names=['Productive', 'Non-productive', 'Neutral']))
  return model

In [ ]:
def classify_url(productive_keywords, non_productive_keywords, model, url):
    metadata = get_metadata_from_url(url)
    clean_content = preprocess_metadata(metadata)
    vectorized_content = vectorizer.transform([clean_content])
    keyword_counts = np.array([[
        count_keywords(clean_content, productive_keywords),
        count_keywords(clean_content, non_productive_keywords)
    ]])
    feature_vector = np.hstack((vectorized_content.toarray(), keyword_counts))
    prediction = model.predict(feature_vector)[0]
    return prediction

In [ ]:
def set_push(dataset_url, url, prediction, flag):
  data = pd.read_csv(dataset_url)
  if flag == 1:
    pred = int(input("Enter Prediction to enlist: 0. Neutral  1. Productive  2. Non-productive : "))
    if pred == 2:
      prediction = "Non-productive"
    elif pred == 0:
      prediction = "Neutral"
    elif pred == 1:
      prediction = "Productive"

  new_row = {'index': len(data) + 1, 'url': url, 'label': prediction}
  new_row_df = pd.DataFrame([new_row])
  data = pd.concat([data, new_row_df], ignore_index=True)
  data.to_csv(dataset_url, index=False)
  print("Row added into Dataset")
  return data

In [ ]:
def add_rows(dataset_url):
    data = pd.read_csv(dataset_url)
    url = input("Enter url to add: ")
    prediction = input("Enter prediction (Productive or Non-productive or Neutral): ")
    new_row = {'index': len(data) + 1, 'url': url, 'label': prediction}
    new_row_df = pd.DataFrame([new_row])
    data = pd.concat([data, new_row_df], ignore_index=True)
    data.to_csv(dataset_url, index=False)
    print("Row added into Dataset")
    return data

In [ ]:
def remove_rows(dataset_url, a, b):
    data = pd.read_csv(dataset_url)
    # Remove rows between indices a and b
    data.drop(data.index[a-1:b], inplace=True)
    # Save the updated DataFrame back to the same CSV file
    data.to_csv(dataset_url, index=False)
    print(f"Rows {a} to {b} removed from Dataset")
    return data

In [ ]:
def display_dataset(dataset_url):
  data = pd.read_csv(dataset_url)
  display(data.tail())

In [ ]:
def final_execute(productive_keywords, non_productive_keywords):
  print("All Productive Keywords:", ", ".join(productive_keywords))
  print("All Non-productive Keywords:", ", ".join(non_productive_keywords))
  dataset_url = '/content/drive/MyDrive/Sayan RP files/Datasets/URL_Dataset(Sheet1).csv'

  while True:
    option = input("Do you want to modify? Y or N : ")
    if ((option == 'Y') or (option == 'y')):
      productive_keywords, non_productive_keywords = keyword_management(productive_keywords, non_productive_keywords)
    else:
        print("All Productive Keywords:", ", ".join(productive_keywords))
        print("All Non-productive Keywords:", ", ".join(non_productive_keywords))
        break

  prod = productive_keywords
  non_prod = non_productive_keywords

  data = prepare_dataset(prod, non_prod, dataset_url)
  model = train_test_model(data)

  new_url = input("Enter url to check for Productive, Non-productive, or Neutral: ")
  classification = classify_url(prod, non_prod, model, new_url)
  print(f"The URL {new_url} is classified as {classification}")

  flag = input("Satisfied with the output? Y or N : ")
  if ((flag == 'Y') or (flag == 'y')):
    push = input('Do you want to add the results to the Dataset? Y or N : ')
    if ((push == 'Y') or (push == 'y')):
      data = set_push(dataset_url, new_url, classification, 0)
      display_dataset(dataset_url)
    else:
      print("Not Added to the Dataset")
      display_dataset(dataset_url)
  else:
    push = input('Do you want to add corrected results to the Dataset? Y or N : ')
    if ((push == 'Y') or (push == 'y')):
      data = set_push(dataset_url, new_url, classification, 1)
      display_dataset(dataset_url)
    else:
      print("Not Added to the Dataset")
      display_dataset(dataset_url)


In [ ]:
def main():
  final_execute(productive_keywords, non_productive_keywords)

In [ ]:
if __name__=="__main__":
    main()

In [ ]:
main()

All Productive Keywords: study, research, education, work, project, python
All Non-productive Keywords: game, social, fun, entertainment, video, reels
Do you want to modify? Y or N : n
All Productive Keywords: study, research, education, work, project, python
All Non-productive Keywords: game, social, fun, entertainment, video, reels
Accuracy: 1.0
Enter url to check for Productive or Non-productive: https://open.spotify.com/album/0LcyzKKw3RjFKL6ygISTeU
The URL https://open.spotify.com/album/0LcyzKKw3RjFKL6ygISTeU is classified as Non-productive
Satisfied with the output? Y or N : Y
Do you want to add the results to the Dataset? Y or N : Y
Row added into Dataset


,index,url,label
6,7,https://www.w3schools.com/python/pandas/pandas...,Productive
7,8,https://stackoverflow.com/questions/13055208/h...,Productive
8,9,https://www.youtube.com/watch?v=J9K3ZZa5gBM&si...,Productive
9,10,https://www.youtube.com/watch?v=QfRHgmEGfvw&li...,Non-productive
10,11,https://open.spotify.com/album/0LcyzKKw3RjFKL6...,Non-productive


In [ ]:
main()

All Productive Keywords: study, research, education, work, project, python
All Non-productive Keywords: game, social, fun, entertainment, video
Do you want to modify? Y or N : y
Enter choice to manage keywords (1 or 2) : 1. Productive  2. Non-productive  3. Display All -> 2
Enter choice to manage non-productive keywords (1 or 2) : 1. Add new  2. Delete old  3. Display all->1
Add NEW -> Operation mode: 1. Single  2. Bulk : 1
Enter new Non-productive Keyword: reels
Do you want to modify? Y or N : n
All Productive Keywords: study, research, education, work, project, python
All Non-productive Keywords: game, social, fun, entertainment, video, reels
Accuracy: 1.0
Enter url to check for Productive, Non-productive, or Neutral: https://www.facebook.com/100044413150303/videos/facebook-reels-expression-update/505333595136542/
The URL https://www.facebook.com/100044413150303/videos/facebook-reels-expression-update/505333595136542/ is classified as Productive
Satisfied with the output? Y or N : n


,index,url,label
7,8,https://stackoverflow.com/questions/13055208/h...,Productive
8,9,https://www.youtube.com/watch?v=J9K3ZZa5gBM&si...,Productive
9,10,https://www.youtube.com/watch?v=QfRHgmEGfvw&li...,Non-productive
10,11,https://open.spotify.com/album/0LcyzKKw3RjFKL6...,Non-productive
11,12,https://www.facebook.com/100044413150303/video...,Non-productive


## Testing Discrepancies

In [ ]:
dataset_url = '/content/drive/MyDrive/Sayan RP files/Datasets/URL_Dataset(Sheet1).csv'
prod = productive_keywords
non_prod = non_productive_keywords

data = prepare_dataset(prod, non_prod, dataset_url)
model = train_test_model(data)

Accuracy: 1.0


In [ ]:
data

,url,label,metadata,clean_content,domain,productive_keyword_count,non_productive_keyword_count
0,https://stackoverflow.com/questions/66341659/e...,Productive,{'title': 'uncaught exception - Electron ipcMa...,uncaught exception electron ipcmain gracefully...,stackoverflow.com,4,0
1,https://www.anandabazar.com/west-bengal/state-...,Non-productive,"{'title': 'Error', 'description': '403 Client ...",error 403 client error forbidden url httpswwwa...,www.anandabazar.com,0,0
2,https://www.barandbench.com/news/sim-swap-scam...,Non-productive,{'title': 'SIM swap scam: NCDRC restores order...,sim swap scam ncdrc restores order directing a...,www.barandbench.com,0,0
3,https://www.youtube.com/watch?v=D_qLCwDiRs4\u0...,Non-productive,{'title': 'East Bengal FC 0 - 1 Mohun Bagan SG...,east bengal fc 0 1 mohun bagan sg final highli...,www.youtube.com,0,0
4,https://www.youtube.com/watch?v=UdcPhnNjSEw,Productive,{'title': 'Collections in Python - Advanced Py...,collections python advanced python 06 programm...,www.youtube.com,6,0
5,https://about.instagram.com/blog/announcements...,Non-productive,"{'title': 'Error', 'description': '400 Client ...",error 400 client error bad request url httpsab...,about.instagram.com,0,0
6,https://www.w3schools.com/python/pandas/pandas...,Productive,"{'title': 'Pandas Read JSON', 'description': '...",pandas read json w3schools offers free online ...,www.w3schools.com,6,3
7,https://stackoverflow.com/questions/13055208/h...,Productive,{'title': 'python - HTTPError: HTTP Error 403:...,python httperror http error 403 forbidden stac...,stackoverflow.com,11,0
8,https://www.youtube.com/watch?v=J9K3ZZa5gBM&si...,Productive,{'title': 'Power BI: Realtime Sales Simulation...,power bi realtime sales simulation using sql s...,www.youtube.com,1,1
9,https://www.youtube.com/watch?v=QfRHgmEGfvw&li...,Non-productive,{'title': 'Shubh - Be Mine (Official Audio) - ...,shubh mine official audio youtube singerlyrics...,www.youtube.com,0,0


In [ ]:
url = 'https://youtu.be/9tZm6ikavlk'

In [ ]:
meta = get_metadata_from_url(url)
meta

{'title': 'Instagram Reels Has Actually Become Tiktok Now - Reels Roast - YouTube',
 'description': 'Today we are watching some very funny reels and shorts.Instagram ► https://www.instagram.com/triggeredinsaanFacebook ► https://www.facebook.com/realtriggered...',
 'image': 'https://i.ytimg.com/vi/9tZm6ikavlk/maxresdefault.jpg',
 'url': 'https://youtu.be/9tZm6ikavlk',
 'text': ''}

In [ ]:
clean = preprocess_metadata(get_metadata_from_url(url))
clean

'instagram reels actually become tiktok reels roast youtube today watching funny reels shortsinstagram ► httpswwwinstagramcomtriggeredinsaanfacebook ► httpswwwfacebookcomrealtriggered'

In [ ]:
domain = extract_domain(url)
domain

'youtu.be'

In [ ]:
prodcount = count_keywords(clean, productive_keywords)
prodcount

0

In [ ]:
nprodcount = count_keywords(clean, non_productive_keywords)
nprodcount

3

In [ ]:
classify_url(productive_keywords, non_productive_keywords, model, url)

'Non-productive'